# LOVIT

## TEXTRANK

In [1]:
from collections import Counter

def scan_vocabulary(sents, tokenize, min_count=2):
    counter = Counter(w for sent in sents for w in tokenize(sent))
    counter = {w:c for w,c in counter.items() if c >= min_count}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

In [2]:
from collections import defaultdict

def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2):
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in tokens_i if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k:v for k,v in counter.items() if v >= min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    return dict_to_mat(counter, n_vocabs, n_vocabs)

In [3]:
from scipy.sparse import csr_matrix

def dict_to_mat(d, n_rows, n_cols):
    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

In [4]:
def word_graph(sents, tokenize=None, min_count=2, window=2, min_cooccurrence=2):
    idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    tokens = [tokenize(sent) for sent in sents]
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence, verbose)
    return g, idx_to_vocab

In [5]:
import numpy as np
from sklearn.preprocessing import normalize

def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)

    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R

In [6]:
def textrank_keyword(sents, tokenize, min_count, window, min_cooccurrence, df=0.85, max_iter=30, topk=30):
    g, idx_to_vocab = word_graph(sents, tokenize, min_count, window, min_cooccurrence)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keywords = [(idx_to_vocab[idx], R[idx]) for idx in reversed(idxs)]
    return keywords

In [7]:
from collections import Counter
from scipy.sparse import csr_matrix
import math

def sent_graph(sents, tokenize, similarity, min_count=2, min_sim=0.3):
    _, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)

    tokens = [[w for w in tokenize(sent) if w in vocab_to_idx] for sent in sents]
    rows, cols, data = [], [], []
    n_sents = len(tokens)
    for i, tokens_i in enumerate(tokens):
        for j, tokens_j in enumerate(tokens):
            if i >= j:
                continue
            sim = similarity(tokens_i, tokens_j)
            if sim < min_sim:
                continue
            rows.append(i)
            cols.append(j)
            data.append(sim)
    return csr_matrix((data, (rows, cols)), shape=(n_sents, n_sents))

def textrank_sent_sim(s1, s2):
    n1 = len(s1)
    n2 = len(s2)
    if (n1 <= 1) or (n2 <= 1):
        return 0
    common = len(set(s1).intersection(set(s2)))
    base = math.log(n1) + math.log(n2)
    return common / base

def cosine_sent_sim(s1, s2):
    if (not s1) or (not s2):
        return 0

    s1 = Counter(s1)
    s2 = Counter(s2)
    norm1 = math.sqrt(sum(v ** 2 for v in s1.values()))
    norm2 = math.sqrt(sum(v ** 2 for v in s2.values()))
    prod = 0
    for k, v in s1.items():
        prod += v * s2.get(k, 0)
    return prod / (norm1 * norm2)

In [9]:
def textrank_keysentence(sents, tokenize, min_count, similarity, df=0.85, max_iter=30, topk=5):
    g = sent_graph(sents, tokenize, min_count, min_sim, similarity)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keysents = [(idx, R[idx], sents[idx]) for idx in reversed(idxs)]
    return keysents

In [10]:
from konlpy.tag import Komoran

komoran = Komoran()
def komoran_tokenize(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

In [45]:
import pandas as pd
# original = pd.read_excel("../코로나19_정제데이터.xlsx", header=None, usecols="C")
original = pd.read_excel("../코로나19_정제데이터.xlsx")
# original
df = original.dropna(axis = 0).reset_index()
df
del(df['index'])
df.columns = ['text']
sents = list(df['text'])
sents
for s in sents:
    s = s.encode('utf-8', 'ignore').decode('utf-8')
    s = s.encode('ascii', 'ignore').decode('ascii')
sents

['폐쇄 업무정지 영업장 손실보상 방안 중앙재난안전대책본부브리핑 지역발생 144명 해외유입 12명 사망 3명 추가 344명 체육 관광 다중이용시설 방역수칙 점검 영세시설 방역물품 추가 지원 예정 중환',
 '광주 신규확진 17명 발생 KBS뉴스 광주 상황 진정 광주 신규확진 17명 3단계 사회적거리두기 시행 광주시 고민',
 '미국 뉴욕 뉴요커 방송 특파원보고세계는지금 세계는지금 국제시사 특파원 세계 매주 토요일 방송 미국 뉴욕 뉴요커',
 '중앙재난안전대책본부 정례브리핑 9월 중앙재난안전대책본부 정례브리핑 9월',
 '중앙재난안전대책본부브리핑 뉴스특보 09월 오늘 오전0시기준 국내신종코로나바이러스감염증신규확진자 119명 누적 2만1천296명 신규확진자 이후 닷새',
 '회복 고통 후유증 3월 완치판정 박현 겸임교수 5개월 후유증 기억 집중 브레인 포그 증상 가슴 복부 통증 피부색 피로감 심해 일상생활 SNS',
 '900만명 대상 인플루엔자 무료 예방접종 브리핑 신규확진 119명 추가 2만1296명 사망 2명추가 336명 서울 도심 집회 관련 5명 추가확진 누적확진 532명 서울 강동구 콜센터 관련',
 '풀영상 중앙재난안전대책본부브리핑 9월 9일 KBS뉴스 국내신규확진자 주일 기록 서울 경기 인천 확진자 제주 강원 경북 제외 전국 14개',
 '청정국 뉴질랜드 비상 총선 연기 앵커 코로나19 대처 평가 청정지역 선언 뉴질랜드 지역감염 발생 오클랜드 지역 봉쇄령',
 '채널A 특집 다큐 코로나 투병 채널A 특집 다큐 코로나 투병 채널A특집다큐 내가겪은코 투병 공식 홈페이지 공식',
 '신규확진 267명 국내발생 253명 해외유입 14명 뉴스특보 09월 오늘 0시기준 국내신종코로나바이러스감염증신규확진자 195명 누적 2만644명 400명 신규확진',
 '확진자 확진 완치 퇴원 코로나 투병 사회적거리두기 코로나 투병 감염 완치 하단 자막 저작권 문제 사람',
 '9월 상업 항체치료제 대량생산계획 브리핑 신규확진136명 추가 2만1432명 격리 339명 늘어나 1

In [46]:
from textrank import KeywordSummarizer

keyword_extractor = KeywordSummarizer(
    tokenize = komoran_tokenize,
    window = -1,
    verbose = False
)

keywords = keyword_extractor.summarize(sents, topk=30)

In [47]:
def komoran_tokenize(sent):
    return komoran.pos(sent, join=True)

keyword_extractor = KeywordSummarizer(tokenize = komoran_tokenize, window = -1)
keywords = keyword_extractor.summarize(sents, topk=30)

In [48]:
def komoran_tokenize(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

keyword_extractor = KeywordSummarizer(tokenize = komoran_tokenize, window = 2)
keywords = keyword_extractor.summarize(sents, topk=30)

In [49]:
from textrank import KeysentenceSummarizer

summarizer = KeysentenceSummarizer(tokenize = komoran_tokenize, min_sim = 0.5)
keysents = summarizer.summarize(sents, topk=10)

In [51]:
summarizer = KeysentenceSummarizer(tokenize = komoran_tokenize, min_sim = 0.3)
keysents = summarizer.summarize(sents, topk=3)

In [52]:
def subword_tokenizer(sent, n=3):
    def subword(token, n):
        if len(token) <= n:
            return [token]
        return [token[i:i+n] for i in range(len(token) - n)]
    return [sub for token in sent.split() for sub in subword(token, n)]

subword_tokenizer('이것은 부분단어의 예시입니다 짧은 어절은 그대로 나옵니다')
# ['이것은', '부분단', '분단어', '단어의', '예시입', '시입니', '입니다', '짧은', '어절은', '그대로', '나옵니', '옵니다']

['이것은', '부분단', '분단어', '예시입', '시입니', '짧은', '어절은', '그대로', '나옵니']

In [53]:
summarizer = KeysentenceSummarizer(tokenize = subword_tokenizer, min_sim = 0.3)
keysents = summarizer.summarize(sents, topk=3)

In [54]:
summarizer.R

array([0.36438105, 0.36462685, 0.32027365, ..., 0.86022459, 0.23879128,
       0.82107415])

In [55]:
import numpy as np

bias = np.ones(len(sents))
bias[-1] = 10
keysents = summarizer.summarize(sents, topk=3, bias=bias)

In [56]:
summarizer.R

array([0.36408808, 0.36436184, 0.31995804, ..., 0.86159543, 0.23859145,
       2.17187683])